# Deducing Struct Layout with Symbolic Execution
The goal of this tutorial is to determine the input format
accepted by the test program `dns.{arch}.elf`
without reading the source or resorting to manual RE.

We'll use memory labelling to lay out a hypothetical message structure
in our machine state, and then use `FieldDetectionAnalysis` to
detect inconsistencies between our hypothesis and how the code accesses the data.
By iterating this process, we will refine our hypothesis until it matches
what the code accepts.

## Setup & Installation
Install the required packages as described in the install directions.

You can build the example programs by running `make all` from this directory.
This requires the cross-compiler toolchains installed as part of the apt dependencies;
this analysis will work on any architecture supported by angr.

Let's test this program against the example input `examples/answer.dat`:

```
$> hexdump -C examples/answer.dat
00000000  ce 70 81 80 00 01 00 01  00 00 00 01 06 67 6f 6f  |.p...........goo|
00000010  67 6c 65 03 63 6f 6d 00  00 01 00 01 c0 0c 00 01  |gle.com.........|
00000020  00 01 00 00 00 d2 00 04  8e fa 41 ce 00 00 29 04  |..........A...).|
00000030  c4 00 00 00 00 00 1c 00  0a 00 18 cb 6e 4b 21 27  |............nK!'|
00000040  ca a2 eb 8f fc 9f 64 67  85 5f 7f cf 3e 38 e4 56  |......dg._..>8.V|
00000050  80 72 f9                                          |.r.|
00000053

$> ./bin/dns.amd64.elf examples/answer.dat
DNS Message:
        TID:          ce70
        FLAGS:        0120
        # Questions:  1
        # Answers:    0
        # Auth. Rs:   0
        # Extra Rs:   1
Question:
        Name:         google.com
        Type:         1
        Class:        1
Record:
        Name:
        Type:         41
        Class:        1232
```

If it's not apparent yet, this program parses basic DNS messages.
However, the exact structure accepted by the parser isn't obvious.
We could decompile the binary and try to elicit the structure but 
we are going to use SmallWord analysis capabilites to figure it out.

## Harnessing the Parser

For sanity's sake, let's assume we know that the parser is implemented in `parse_dns_msg()`,
and that we roughly know its arguments from `main()`:

- **Arg 1:** `uint8_t *` pointing to an input buffer
- **Arg 2:** `size_t` holding its size
- **Arg 3:** `size_t *` pointing to a value starting at zero
- **Arg 4:** `struct *` pointing to an unknown struct

Our goal for this exercise is to figure out the structure of argument 1,
which we'll call `buf`, and argument 4, which we'll call `msg`.

First, we set up some smallworld boilerplate.
This configures logging, as well as the hinter
that will let us see the output from our analysis. 

In [1]:
import warnings

warnings.filterwarnings(
    "ignore", 
    message="Protobuf gencode version .* is exactly one major version older.*"
)

In [2]:
import logging
import pathlib

import smallworld
import smallworld.analyses.field_detection
import smallworld.analyses.unstable.angr.visitor
from smallworld.analyses.field_detection import FieldDetectionAnalysis

# Set up logging and hinting                                                                                                                                                                                                         
smallworld.logging.setup_logging(level=logging.INFO)
smallworld.hinting.setup_hinting(stream=True, verbose=True)

log = logging.getLogger("smallworld")

Next, we create the basic machine state, load our ELF file into a smallworld state object, and add the code to the machine.

Since we don't expect a specific platform, we can use the platform definition parsed out of the ELF headers.

Also, since this is the only ELF we're loading, we can use its notion of program bounds to define which memory the analysis can execute.

In [3]:
# Create a machine
machine = smallworld.state.Machine()

# Load the code
filepath = "bin/dns.amd64.elf"
with open(filepath, "rb") as f:
    code = smallworld.state.memory.code.Executable.from_elf(f, 0x40000)
    machine.add(code)

# Apply the code's bounds to the machine
for bound in code.bounds:
    machine.add_bound(bound[0], bound[1])

# Use the ELF's notion of the platform
platform = code.platform

 # Create the analysis; we'll need it later.
analysis = FieldDetectionAnalysis(platform)

[+] Address: 40000
[+] {"time": "2025-08-07 14:50:25,323", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies interpreter b'/lib64/ld-linux-x86-64.so.2\\x00'", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:50:25,324", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program includes dynamic linking metadata", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:50:25,325", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies stack permissions", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:50:25,327", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies RELRO data", "class": "smallworld.hinting.hinting.Hint"}}


In [5]:
# Create a CPU
cpu = smallworld.state.cpus.CPU.for_platform(platform)
machine.add(cpu)

# Use lief to find the address of parse_dns_message
sym = code.get_symbol_value("parse_dns_message")
cpu.rip.set(sym)

# Add a blank stack
stack = smallworld.state.memory.stack.Stack.for_platform(
    platform, 0x7FFFFFFFC000, 0x4000
)
machine.add(stack)

stack.push_integer(0x01010101, 8, None)

# Configure the stack pointer
sp = stack.get_pointer()
cpu.rsp.set(sp)


Now that we have the program set up, we can start to model the inputs to `parse_dns_message()`.
We know we need a message struct, a memory buffer, and a `size_t`
that are all passed by reference.

For ease of modelling, let's just allocate them all as globals.

To start, we'll allocate `buf` and `msg` as blank swaths of memory.
As we learn more about their internal structure from the analysis,
we will come back and refine this part of our harness.

In [6]:
# Configure somewhere for arguments to live                                                                                                                                                                                          
gdata = smallworld.state.memory.Memory(0x6000, 0x1000)
machine.add(gdata)
# DNS message struct                                                                                                                                                                                                                 
# Sort of cheating that I know how big it is.                                                                                                                                                                                        
gdata[0] = smallworld.state.SymbolicValue(48, None, None, "msg")
# Input buffer                                                                                                                                                                                                                       
gdata[48] = smallworld.state.SymbolicValue(512, None, None, "buf")
# Offset into buffer                                                                                                                                                                                                                 
gdata[560] = smallworld.state.IntegerValue(0, 8, "off", False)


Next, we use what we know about the function signature to assign values to registers.
We know that the first arg points to `buf`, the second is the size of `buf`,
the third points to `off`, and the fourth points to `msg`.

In [7]:
# Configure arguments                                                                                                                                                                                                                
# arg 0: pointer to buf                                                                                                                                                                                                              
cpu.rdi.set(gdata.address + 48)
cpu.rdi.set_label("PTR buf")
# arg 1: buffer capacity                                                                                                                                                                                                             
cpu.rsi.set(512)
cpu.rsi.set_label("cap")
# arg 2: pointer to off                                                                                                                                                                                                              
cpu.rdx.set(gdata.address + 560)
cpu.rdx.set_label("PTR off")
# arg 3: pointer to msg                                                                                                                                                                                                              
cpu.rcx.set(gdata.address)
cpu.rcx.set_label("PTR msg")

At long last, we're reading to actually run our analysis.

In [8]:
machine.analyze(FieldDetectionAnalysis(platform))

[!] 0x7fffffffc000 - 0x800000000000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6030: <smallworld.state.state.SymbolicValue object at 0x782901109090> := msg
[+] {"time": "2025-08-07 14:50:43,872", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24576, "size": 48, "label": "msg", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6030 - 0x6230: <smallworld.state.state.SymbolicValue object at 0x78290110b310> := buf
[+] {"time": "2025-08-07 14:50:43,874", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24624, "size": 512, "label": "buf", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6230 - 0x6238: <smallworld.state.state.IntegerValue object at 0x782901109060> := off
[+] {"time": "2025-08-07 14:50:43,875", "level": "INFO", "source": "smallworld.analyses.field_

# First Results

(This code so far is collected in `dns_0.py`.)  The analysis produced by `FieldDetectionAnalysis` yields the following hint:

```
[*] Partial write to known field
[*]   address:    0x6010
[*]   size:       8
[*]   pc:         0x41e73
[*]   access:     write
[*]   guards:
[*]   field:      msg[16:24]
[*]   expr:       <BV64 0x0>
```
The code at `0x41e73` tried to write 8 bytes of zero starting 16 bytes into our `msg` buffer.
But we defined `msg` as a 512 byte contiguous region and didn't expect to see a field.  

With this information in hand we can redefine `msg` as the following


In [9]:
gdata[0] = smallworld.state.SymbolicValue(16, None, None, "msg.unk-0x0")
gdata[16] = smallworld.state.SymbolicValue(8, None, None, "msg.a")
gdata[24] = smallworld.state.SymbolicValue(24, None, None, "msg.unk-0x18")

We're not sure what `msg.a` is, but we know it's very likely a field.
We're on our way!

Running our updated script gives a very similar hint to the first:

In [10]:
machine.analyze(FieldDetectionAnalysis(platform))

[!] 0x7fffffffc000 - 0x800000000000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6010: <smallworld.state.state.SymbolicValue object at 0x782938192770> := msg.unk-0x0
[+] {"time": "2025-08-07 14:50:53,611", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24576, "size": 16, "label": "msg.unk-0x0", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6030 - 0x6230: <smallworld.state.state.SymbolicValue object at 0x78290110b310> := buf
[+] {"time": "2025-08-07 14:50:53,612", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24624, "size": 512, "label": "buf", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6230 - 0x6238: <smallworld.state.state.IntegerValue object at 0x782901109060> := off
[+] {"time": "2025-08-07 14:50:53,614", "level": "INFO", "source": "smallworld

Running our updated script gives a very similar hint to the first:

```
[*] Partial write to known field
[*]   address:    0x6018
[*]   size:       8
[*]   pc:         0x41e7f
[*]   access:     write
[*]   guards:
[*]   field:      msg.unk.0x18[0:8]
[*]   expr:       <BV64 0x0>
```

In fact, four iterations tell us the same thing;
the program writes zero to four 8-byte fields right next to each other:


In [11]:
gdata[0] = smallworld.state.SymbolicValue(16, None, None, "msg.unk-0x0")
gdata[16] = smallworld.state.SymbolicValue(8, None, None, "msg.a")
gdata[24] = smallworld.state.SymbolicValue(8, None, None, "msg.b")
gdata[32] = smallworld.state.SymbolicValue(8, None, None, "msg.c")
gdata[40] = smallworld.state.SymbolicValue(8, None, None, "msg.d")


Some more iterations, and we start decoding `msg.unk-0x0`.

In [12]:
machine.analyze(FieldDetectionAnalysis(platform))

[!] 0x7fffffffc000 - 0x800000000000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6010: <smallworld.state.state.SymbolicValue object at 0x7828ff7cbf40> := msg.unk-0x0
[+] {"time": "2025-08-07 14:50:59,203", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24576, "size": 16, "label": "msg.unk-0x0", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6030 - 0x6230: <smallworld.state.state.SymbolicValue object at 0x78290110b310> := buf
[+] {"time": "2025-08-07 14:50:59,204", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24624, "size": 512, "label": "buf", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6230 - 0x6238: <smallworld.state.state.IntegerValue object at 0x782901109060> := off
[+] {"time": "2025-08-07 14:50:59,205", "level": "INFO", "source": "smallworld

Iterating on this process we find that we have multiple 2 byte regions which we identify as parts of `msg.hdr`, since we suspect it's the part of the struct representing the DNS header.  The information in the final 4 bytes of this region are never referenced.  We leave them as a final 4 byte region we call `msg.hdr.unused`:

In [13]:
gdata[0] = smallworld.state.SymbolicValue(2, None, None, "msg.hdr.a")
gdata[2] = smallworld.state.SymbolicValue(2, None, None, "msg.hdr.b")
gdata[4] = smallworld.state.SymbolicValue(2, None, None, "msg.hdr.c")
gdata[6] = smallworld.state.SymbolicValue(2, None, None, "msg.hdr.d")
gdata[8] = smallworld.state.SymbolicValue(2, None, None, "msg.hdr.e")
gdata[10] = smallworld.state.SymbolicValue(2, None, None, "msg.hdr.f")
gdata[12] = smallworld.state.SymbolicValue(4, None, None, "msg.hdr.unused")
machine.analyze(FieldDetectionAnalysis(platform))

[!] 0x7fffffffc000 - 0x800000000000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6002: <smallworld.state.state.SymbolicValue object at 0x7828ff855bd0> := msg.hdr.a
[+] {"time": "2025-08-07 14:51:04,310", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24576, "size": 2, "label": "msg.hdr.a", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6030 - 0x6230: <smallworld.state.state.SymbolicValue object at 0x78290110b310> := buf
[+] {"time": "2025-08-07 14:51:04,311", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24624, "size": 512, "label": "buf", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6230 - 0x6238: <smallworld.state.state.IntegerValue object at 0x782901109060> := off
[+] {"time": "2025-08-07 14:51:04,314", "level": "INFO", "source": "smallworld.anal

Repeating this process for `buf` we can further identify regions which we summarize as:

In [14]:
gdata[48] = smallworld.state.SymbolicValue(2, None, None, "buf.a")
gdata[50] = smallworld.state.SymbolicValue(2, None, None, "buf.b")
gdata[52] = smallworld.state.SymbolicValue(2, None, None, "buf.c")
gdata[54] = smallworld.state.SymbolicValue(2, None, None, "buf.d")
gdata[56] = smallworld.state.SymbolicValue(2, None, None, "buf.e")
gdata[58] = smallworld.state.SymbolicValue(2, None, None, "buf.f")
gdata[60] = smallworld.state.SymbolicValue(500, None, None, "buf")
machine.analyze(FieldDetectionAnalysis(platform))

[!] 0x7fffffffc000 - 0x800000000000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6002: <smallworld.state.state.SymbolicValue object at 0x7828ff855bd0> := msg.hdr.a
[+] {"time": "2025-08-07 14:51:08,390", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24576, "size": 2, "label": "msg.hdr.a", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6030 - 0x6032: <smallworld.state.state.SymbolicValue object at 0x7828ff8b4b80> := buf.a
[+] {"time": "2025-08-07 14:51:08,392", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24624, "size": 2, "label": "buf.a", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6230 - 0x6238: <smallworld.state.state.IntegerValue object at 0x782901109060> := off
[+] {"time": "2025-08-07 14:51:08,392", "level": "INFO", "source": "smallworld.an

The work above is summarized in `dns_1.py`.



# Adding malloc

Notice that the `FieldDetectionAnalysis` routine no longer provides information about a partial access to a known field, so execution continues.  The last successful read is reported to be at instruction `0x41ed2`, and that the analysis ends there with the last known good state being reported as:

```
[!] State at <BV64 0x1076>:
```

We loaded our ELF at 0x40000 so something's way off.
If we disassemble the code and look around 0x1ed2 (which is 0x41ed2 minus the loading address 0x40000) we find:

```
$> objdump -d -R ./bin/dns.amd64.elf
...
    1ed2:       0f b7 40 04             movzwl 0x4(%rax),%eax
    1ed6:       48 69 f8 08 01 00 00    imul   $0x108,%rax,%rdi
    1edd:       e8 8e f1 ff ff          call   1070 <malloc@plt>
...
```

This is an external call to `malloc()` which we have not loaded.

We have not yet discussed how to handle an external call like this.  One strategey would be to stub out `malloc()` and carry on.  Howerver, we would 
like to know what is happening inside the heap-allocated buffers as well.
Luckily, `FieldDetectionAnalysis` comes with a malloc model.
Let's update our harness:

Note: At this point we recreate our machine so any changes we made along the way are reset.

In [15]:
def reinitialize_demo():
    # reinitializing outside of this routine
    global machine
    global platform
    global analysis
    global cpu
    global sym
    global stack
    global sp
    global code


    # Create a machine
    machine = smallworld.state.Machine()

    # Load the code
    filepath = "bin/dns.amd64.elf"
    with open(filepath, "rb") as f:
        code = smallworld.state.memory.code.Executable.from_elf(f, 0x40000)
        machine.add(code)

    # Apply the code's bounds to the machine
    for bound in code.bounds:
       machine.add_bound(bound[0], bound[1])

    # Use the ELF's notion of the platform
    platform = code.platform

    # Create the analysis; we'll need it later.
    analysis = FieldDetectionAnalysis(platform)

    # Create a CPU
    cpu = smallworld.state.cpus.CPU.for_platform(platform)
    machine.add(cpu)

    # Use lief to find the address of parse_dns_message
    sym = code.get_symbol_value("parse_dns_message")
    cpu.rip.set(sym)

    # Add a blank stack
    stack = smallworld.state.memory.stack.Stack.for_platform(
        platform, 0x7FFFFFFFC000, 0x4000
    )
    machine.add(stack)
    machine.add(gdata)

    stack.push_integer(0x01010101, 8, None)

    # Configure the stack pointer
    sp = stack.get_pointer()
    cpu.rsp.set(sp)

    # Configure arguments
    # arg 0: pointer to buf
    cpu.rdi.set(gdata.address + 48)
    cpu.rdi.set_label("PTR buf")
    # arg 1: buffer capacity
    cpu.rsi.set(512)
    cpu.rsi.set_label("cap")
    # arg 2: pointer to off
    cpu.rdx.set(gdata.address + 560)
    cpu.rdx.set_label("PTR off")
    # arg 3: pointer to msg
    cpu.rcx.set(gdata.address)
    cpu.rcx.set_label("PTR msg")
reinitialize_demo()

[+] Address: 40000
[+] {"time": "2025-08-07 14:51:21,294", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies interpreter b'/lib64/ld-linux-x86-64.so.2\\x00'", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:51:21,295", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program includes dynamic linking metadata", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:51:21,296", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies stack permissions", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:51:21,296", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies RELRO data", "class": "smallworld.hinting.hinting.Hint"}}


In [16]:
from smallworld.analyses.field_detection import MallocModel, FreeModel
# Add a blank heap 
heap = smallworld.state.memory.heap.BumpAllocator(0x20000, 0x10000)
machine.add(heap)

# Configure malloc and free models
malloc = MallocModel(
    0x10000, heap, platform, analysis.mem_read_hook, analysis.mem_write_hook
)
machine.add(malloc)
machine.add_bound(malloc._address, malloc._address + 16)

free = FreeModel(0x1036)
machine.add(free)
machine.add_bound(free._address, free._address + 16)


This will invoke a function hook mimicking `malloc()` if we call address 0x10000.
If we were in a raw binary, we'd have to do some manual patching
to tell it where we put `malloc()`, but this is an ELF.
Let's use the relocation entries to our advantage:

In [17]:
# Apply relocations to malloc
code.update_symbol_value("malloc", malloc._address)

So we can continue our analysis here or if you want to run outside this notebook the worke we have done up to this point is fully contained in `dns_2.py`:

In [18]:
machine.analyze(analysis)

[!] 0x7fffffffc000 - 0x800000000000
[!] 0x20000 - 0x30000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6002: <smallworld.state.state.SymbolicValue object at 0x7828ff855bd0> := msg.hdr.a
[+] {"time": "2025-08-07 14:51:26,709", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24576, "size": 2, "label": "msg.hdr.a", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6030 - 0x6032: <smallworld.state.state.SymbolicValue object at 0x7828ff8b4b80> := buf.a
[+] {"time": "2025-08-07 14:51:26,712", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24624, "size": 2, "label": "buf.a", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6230 - 0x6238: <smallworld.state.state.IntegerValue object at 0x782901109060> := off
[+] {"time": "2025-08-07 14:51:26,714", "level": "INFO", "s

This ran for much longer, but eventually halted with an error similar to this:

```
[*] Capacity did not collapse to one value
[*] The following fields are lengths:
[*]    msg.hdr.c_27_16
[*] Concretize them to continue analysis
```

This output indicates that the analysis was able to deduce that the `msg.hdr.c` field
gives the numbe of items allocated in our buffer.  In fact, if we scroll up to instruction 
0x418d0 we see that `msg.hdr.c` gets filled in by the contents of `buf.c` so if we want to
concretize `msg.hdr.c` we need to assign a concrete value to `buf.c` (and give it a new name):

In [17]:
gdata[52] = smallworld.state.BytesValue(b"\x00\x01", "buf.msg.hdr.c")
machine.analyze(FieldDetectionAnalysis(platform))

[!] 0x7fffffffc000 - 0x800000000000
[!] 0x40000 - 0x46000
[!] 0x20000 - 0x30000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6002: <smallworld.state.state.SymbolicValue object at 0x7a130797f280> := msg.hdr.a
[+] {"time": "2025-07-08 09:59:53,543", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24576, "size": 2, "label": "msg.hdr.a", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6030 - 0x6032: <smallworld.state.state.SymbolicValue object at 0x7a130784b4c0> := buf.a
[+] {"time": "2025-07-08 09:59:53,545", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24624, "size": 2, "label": "buf.a", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6230 - 0x6238: <smallworld.state.state.IntegerValue object at 0x7a133c3a19c0> := off
[+] {"time": "2025-07-08 09:59:53,546

## Analyzing the behavior of malloc

Scrolling up to the malloc call after the memory read at 0x41ed2 we see the following output from the malloc:

```
[!] malloc: Alloc'd 264 bytes at 0x20010
[+] Storing 8 bytes at 0x20008
[!]   Length field msg.hdr.c not known
[+] 0x41ee6: WRITING 0x6010 - 0x6018 (msg.a)
[+]   <BV64 0x20010>
```

Let's take this one line at a time:

- `malloc: Alloc'd 264 bytes at 0x20010`:  Since we assigned 0x1 to the length in `buf.c`, 
this line is telling us that space for the struct is 264 bytes (the `malloc()` model would 
warn us if the length computation was more complex than "variable * magnitude".)

- `Storing 8 bytes at 0x20008`: Malloc returns a pointer to the newly allocated memory so the address of the heap memory is stored there. #TODO - confirm this is true?

- `  Length field msg.hdr.c not known`: We haven't told the `malloc()` model that it should track structs allocated using `msg.hdr.c`, or what they should look like. #TODO - review this comment, I don't fully understand it

- `0x41ee6: WRITING 0x6010 - 0x6018 (msg.a)`: The pointer to our new heap array is stored in `msg.a`, making `msg.hdr.c` the length of `msg.a`.

We wrap up this section by noting that execution stopped on another read of a single byte from a larger buffer.  We'll finish this treatment of models with three final actions:
- combine all the model work into a routine that can be called when we reinitialize our analysis
- update our harness to take account of the single byte
- run again

In [22]:
def add_malloc_free_models():
    global code
    global malloc
    global free
    global heap

    # Add a blank heap
    heap = smallworld.state.memory.heap.BumpAllocator(0x20000, 0x10000)
    machine.add(heap)

    # Configure malloc and free models
    malloc = MallocModel(
        0x10000, heap, platform, analysis.mem_read_hook, analysis.mem_write_hook
    )
    machine.add(malloc)
    machine.add_bound(malloc._address, malloc._address + 16)

    free = FreeModel(0x1036)
    machine.add(free)
    machine.add_bound(free._address, free._address + 16)

    # Apply relocations to malloc
    code.update_symbol_value("malloc", malloc._address)
  
reinitialize_demo()
add_malloc_free_models()
gdata[60] = smallworld.state.SymbolicValue(1, None, None, "buf.g")
gdata[61] = smallworld.state.SymbolicValue(499, None, None, "buf")
machine.analyze(analysis)

[+] Address: 40000
[+] {"time": "2025-08-07 14:55:52,117", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies interpreter b'/lib64/ld-linux-x86-64.so.2\\x00'", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:55:52,117", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program includes dynamic linking metadata", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:55:52,118", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies stack permissions", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:55:52,119", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies RELRO data", "class": "smallworld.hinting.hinting.Hint"}}
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6002: <smallworld.state.state.SymbolicValue object at 0x7828ff855bd0> := msg.hd

# Stage 2 DNS exploration: Next control field

The end of our last run produced new output.  In particular, it noted that `buf.g` seems to be
influencing whether the first two bytes of `buf` are a single field or multiple fields.  What
conditions are making this happen?  To address this question, we look at the various reports 
of `Partial access to known field` and look at the conditions that were required for that 
partial access to occurr.  This happens in three cases.  Extracting just the information 
relevent to `buf.g` we have:

### Case 1:
```
[*] Partial access to known field
[*]   address:    0x603d
[*]   size:       1
[*]   pc:         0x415c8
[*]   access:     read
[*]   guards:
[*]     0x41592: <Bool buf.g != 0> [[<BV8 buf.g>]]
[*]     0x415ab: <Bool (buf.g & 192) == 192> [[<BV8 buf.g>]]
[*]   field:      buf[0:1]
```
We get to this case exactly when the two high bits set on `buf.g`.  So since `buf.g` is a byte, 
`buf.g` is 64 and 255 inclusive.

### Case 2:
```
[*] Partial access to known field
[*]   address:    0x603d
[*]   size:       2
[*]   pc:         0x41a44
[*]   access:     read
[*]   guards:
[*]     0x41592: <Bool buf.g == 0> [[<BV8 buf.g>]]
[*]   field:      buf[0:2]
```
We get to this case exactly when `buf.g == 0`.

### Case 3:
```
[*] Partial access to known field
[*]   address:    0x603d
[*]   size:       1
[*]   pc:         0x416b9
[*]   access:     read
[*]   guards:
[*]     0x41592: <Bool buf.g != 0> [[<BV8 buf.g>]]
[*]     0x415ab: <Bool (buf.g & 192) != 192> [[<BV8 buf.g>]]
[*]     0x4161c: <Bool (buf.g & 192) == 0> [[<BV8 buf.g>]]
[*]     0x41638: <Bool ((if (0#32 .. (0#24 .. buf.g)) < 0xff then 1 else 0) | (if buf.g == 255 then 1 else 0)) != 0> [[<BV8 buf.g>]]
[*]     0x4165c: <Bool cap - off_184_64 >= (0#32 .. (0#24 .. buf.g))> [[<BV8 buf.g>, <BV64 off_184_64>, <BV64 cap>]]
[*]     0x416a4: <Bool 0x0 < (0#32 .. (0#24 .. buf.g))> [[<BV8 buf.g>]]
[*]   field:      buf[0:1]
```
The conditions on `buf.g` look much more complicated here but with a little work 
it is not hard to see that this case arises exactly when the two high bits are
unset on `buf.g` and `buf.g` non-zero.  So since `buf.g` is a byte, `buf.g` is 
between 1 and 63 inclusive.

## Trying some concrete examples

So can we try setting `buf.g` to some concrete values?  Of course, let's try a few:

### Case 1: `64 <= buf.g <= 255`

Let's try setting the buffer to 64 and see what happens.  Having used symbolic execution
in the past, we wrap our analysis with a timeout to catch in case it runs forever!

In [24]:
from smallworld.analyses.field_detection import MallocModel, FreeModel
import claripy
import timeout_decorator

@timeout_decorator.timeout(40)  # argument is in seconds
def analyze_with_timeout(machine, analysis):
    try:
        machine.analyze(analysis)
    except timeout_decorator.timeout_decorator.TimeoutError:
        print("Analysis timeout")


reinitialize_demo()
add_malloc_free_models()
gdata[61] = smallworld.state.SymbolicValue(1, None, None, "buf.one")
gdata[62] = smallworld.state.SymbolicValue(498, None, None, "buf")

# Add constraint to buf.g
machine.add_constraint(
    claripy.UGT(claripy.BVS("buf.g", 8, explicit_name=True), claripy.BVV(64, 8))
)

analyze_with_timeout(machine, analysis)


[+] Address: 40000
[+] {"time": "2025-08-07 14:56:56,768", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies interpreter b'/lib64/ld-linux-x86-64.so.2\\x00'", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:56:56,770", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program includes dynamic linking metadata", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:56:56,770", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies stack permissions", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:56:56,772", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies RELRO data", "class": "smallworld.hinting.hinting.Hint"}}
[!] 0x40000 - 0x46000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6002: <smallworld.state.state.SymbolicValue object at 0x7

Notice that this has run to `0x1010101` which is the return value from this subroutine.  In other words, this exectuion has run to completeion.  Further subdividing `buf.a` into `buf.a1` and `buf.a2` we have:

In [25]:
reinitialize_demo()
add_malloc_free_models()
gdata[48] = smallworld.state.SymbolicValue(1, None, None, "buf.a1")
gdata[49] = smallworld.state.SymbolicValue(1, None, None, "buf.a2")

# Add constraint to buf.g
machine.add_constraint(
    claripy.UGT(claripy.BVS("buf.g", 8, explicit_name=True), claripy.BVV(64, 8))
)

analyze_with_timeout(machine, analysis)

[+] Address: 40000
[+] {"time": "2025-08-07 14:57:13,255", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies interpreter b'/lib64/ld-linux-x86-64.so.2\\x00'", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:57:13,256", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program includes dynamic linking metadata", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:57:13,257", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies stack permissions", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:57:13,258", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies RELRO data", "class": "smallworld.hinting.hinting.Hint"}}
[!] 0x40000 - 0x46000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6002: <smallworld.state.state.SymbolicValue object at 0x7

And iterating a few times:

In [26]:
reinitialize_demo()
add_malloc_free_models()
gdata[4] = smallworld.state.SymbolicValue(1, None, None, "buf.msg.hdr.c1")
gdata[5] = smallworld.state.SymbolicValue(1, None, None, "buf.msg.hdr.c2")
gdata[50] = smallworld.state.SymbolicValue(1, None, None, "buf.msg.hdr.b1")
gdata[51] = smallworld.state.SymbolicValue(1, None, None, "buf.msg.hdr.b2")
gdata[56] = smallworld.state.SymbolicValue(1, None, None, "buf.msg.hdr.e1")
gdata[57] = smallworld.state.SymbolicValue(1, None, None, "buf.msg.hdr.e2")
gdata[61] = smallworld.state.SymbolicValue(1, None, None, "buf.1")
gdata[62] = smallworld.state.SymbolicValue(1, None, None, "buf.2")
gdata[63] = smallworld.state.SymbolicValue(1, None, None, "buf.3")
gdata[64] = smallworld.state.SymbolicValue(1, None, None, "buf.4")
gdata[65] = smallworld.state.SymbolicValue(495, None, None, "buf.rest")
# Add constraint to buf.g
machine.add_constraint(
    claripy.UGT(claripy.BVS("buf.g", 8, explicit_name=True), claripy.BVV(64, 8))
)

analyze_with_timeout(machine, analysis)

[+] Address: 40000
[+] {"time": "2025-08-07 14:57:16,785", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies interpreter b'/lib64/ld-linux-x86-64.so.2\\x00'", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:57:16,786", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program includes dynamic linking metadata", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:57:16,787", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies stack permissions", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:57:16,788", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies RELRO data", "class": "smallworld.hinting.hinting.Hint"}}
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6002: <smallworld.state.state.SymbolicValue object at 0x7828ff855bd0> := msg.hd

We are again asked to concretize a value, this time `msg.hdr.d`.  

In [27]:
gdata[54] = smallworld.state.BytesValue(b"\x00\x01", "buf.msg.hdr.d")
reinitialize_demo()
add_malloc_free_models()
machine.add_constraint(
    claripy.UGT(claripy.BVS("buf.g", 8, explicit_name=True), claripy.BVV(64, 8))
)
analyze_with_timeout(machine, analysis)

[+] Address: 40000
[+] {"time": "2025-08-07 14:57:24,260", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies interpreter b'/lib64/ld-linux-x86-64.so.2\\x00'", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:57:24,261", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program includes dynamic linking metadata", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:57:24,262", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies stack permissions", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-08-07 14:57:24,262", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies RELRO data", "class": "smallworld.hinting.hinting.Hint"}}
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6002: <smallworld.state.state.SymbolicValue object at 0x7828ff855bd0> := msg.hd